# Fujii Kaze's tracks extraction and transformation

In [2]:
# import library
import pandas as pd

## Get 50 tracks by artist's name

### Extract [Spotify API]

In [3]:
# call myFunctions.py file
%run myFunctions.py

token = get_token()

# call function get_song() to extract track data of the artist via spotify api
searched_name = input("Enter artist name: ")
songs = get_song(token, searched_name)

# store the retrived data in the dataframe
columns = list(songs[0].keys())
rows = []
for row in songs:
    rows.append(list(row.values()))
df = pd.DataFrame(data=rows, columns=columns)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   album          50 non-null     object
 1   artists        50 non-null     object
 2   disc_number    50 non-null     int64 
 3   duration_ms    50 non-null     int64 
 4   explicit       50 non-null     bool  
 5   external_ids   50 non-null     object
 6   external_urls  50 non-null     object
 7   href           50 non-null     object
 8   id             50 non-null     object
 9   is_local       50 non-null     bool  
 10  is_playable    50 non-null     bool  
 11  name           50 non-null     object
 12  popularity     50 non-null     int64 
 13  preview_url    0 non-null      object
 14  track_number   50 non-null     int64 
 15  type           50 non-null     object
 16  uri            50 non-null     object
dtypes: bool(3), int64(4), object(10)
memory usage: 5.7+ KB


In [5]:
# explore each column's value
df.iloc[0]

album            {'album_type': 'single', 'artists': [{'externa...
artists          [{'external_urls': {'spotify': 'https://open.s...
disc_number                                                      1
duration_ms                                                 310997
explicit                                                     False
external_ids                              {'isrc': 'JPPO02400480'}
external_urls    {'spotify': 'https://open.spotify.com/track/4h...
href             https://api.spotify.com/v1/tracks/4hsEFcCA7AL5...
id                                          4hsEFcCA7AL5hlKG4PRp2Z
is_local                                                     False
is_playable                                                   True
name                                   Michi Teyu Ku (Overflowing)
popularity                                                      69
preview_url                                                   None
track_number                                                  

In [6]:
# explore album object which is another entity
df["album"][0]

{'album_type': 'single',
 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6bDWAcdtVR3WHz2xtiIPUi'},
   'href': 'https://api.spotify.com/v1/artists/6bDWAcdtVR3WHz2xtiIPUi',
   'id': '6bDWAcdtVR3WHz2xtiIPUi',
   'name': 'Fujii Kaze',
   'type': 'artist',
   'uri': 'spotify:artist:6bDWAcdtVR3WHz2xtiIPUi'}],
 'external_urls': {'spotify': 'https://open.spotify.com/album/06qRSgIo9l4Gf7ACV6JZQq'},
 'href': 'https://api.spotify.com/v1/albums/06qRSgIo9l4Gf7ACV6JZQq',
 'id': '06qRSgIo9l4Gf7ACV6JZQq',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/ab67616d0000b27336f72e16774f67f6a4df173d',
   'width': 640},
  {'height': 300,
   'url': 'https://i.scdn.co/image/ab67616d00001e0236f72e16774f67f6a4df173d',
   'width': 300},
  {'height': 64,
   'url': 'https://i.scdn.co/image/ab67616d0000485136f72e16774f67f6a4df173d',
   'width': 64}],
 'is_playable': True,
 'name': 'Michi Teyu Ku (Overflowing)',
 'release_date': '2024-03-14',
 'release_date_precision': 'day',

In [7]:
# explore artists object which is another entity
df["artists"][0]

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/6bDWAcdtVR3WHz2xtiIPUi'},
  'href': 'https://api.spotify.com/v1/artists/6bDWAcdtVR3WHz2xtiIPUi',
  'id': '6bDWAcdtVR3WHz2xtiIPUi',
  'name': 'Fujii Kaze',
  'type': 'artist',
  'uri': 'spotify:artist:6bDWAcdtVR3WHz2xtiIPUi'}]

### Transform

In [8]:
# in this project, we extraxt only the album_id, album_name, and release_date from the album object
albums_id = []
albums_name = []
release_date = []
type = []
for i in range(len(df)):
    albums_id.append(df.iloc[i]["album"]["id"])
    albums_name.append(df.iloc[i]["album"]["name"])
    release_date.append(df.iloc[i]["album"]["release_date"])
    type.append(df.iloc[i]["album"]["type"])


album_id_series = pd.Series(albums_id , name='album_id')
album_name_series = pd.Series(albums_name, name='album_name')
release_date_series = pd.Series(release_date, name='release_date')
type_series = pd.Series(type, name='type')

df["album_id"] = album_id_series
df["album_name"] = album_name_series
df["release_date"] = release_date_series
df["type"] = type_series 


In [9]:
# rename the id and name to be track_id and track_name
df = df.rename(columns={"id": "track_id", "name": "track_name"})

In [10]:
# selected columns
track_data = df[["track_id", "track_name","href","popularity","uri","release_date", "album_id", "album_name"]]

In [11]:
track_data.head()

,track_id,track_name,href,popularity,uri,release_date,album_id,album_name
0,4hsEFcCA7AL5hlKG4PRp2Z,Michi Teyu Ku (Overflowing),https://api.spotify.com/v1/tracks/4hsEFcCA7AL5...,69,spotify:track:4hsEFcCA7AL5hlKG4PRp2Z,2024-03-14,06qRSgIo9l4Gf7ACV6JZQq,Michi Teyu Ku (Overflowing)
1,0o9zmvc5f3EFApU52PPIyW,Shinunoga E-Wa,https://api.spotify.com/v1/tracks/0o9zmvc5f3EF...,78,spotify:track:0o9zmvc5f3EFApU52PPIyW,2020-05-20,1OojCidx0eoPKch2M0Kz31,HELP EVER HURT NEVER
2,7AMGgAPFczs3wJgMqu6Eqi,Matsuri,https://api.spotify.com/v1/tracks/7AMGgAPFczs3...,68,spotify:track:7AMGgAPFczs3wJgMqu6Eqi,2022-03-23,7Ip9X7pnkhJ4cwDoBnvneD,LOVE ALL SERVE ALL
3,0oz4Oh0zx4XRZnS6Z1rr0M,きらり,https://api.spotify.com/v1/tracks/0oz4Oh0zx4XR...,0,spotify:track:0oz4Oh0zx4XRZnS6Z1rr0M,2024-05-28,1DeciVpwShHj82dFZJiO0N,Best of Fujii Kaze 2020-2024
4,53GYyXuqqC1ZBLBvgXA1QW,Workin' Hard,https://api.spotify.com/v1/tracks/53GYyXuqqC1Z...,0,spotify:track:53GYyXuqqC1ZBLBvgXA1QW,2024-05-28,1DeciVpwShHj82dFZJiO0N,Best of Fujii Kaze 2020-2024


In [12]:
# check null values
track_data.isnull().sum()

track_id        0
track_name      0
href            0
popularity      0
uri             0
release_date    0
album_id        0
album_name      0
dtype: int64

In [13]:
# check duplications
track_data.duplicated().sum()

0

## Get track's audio analysis

### Extract [Spotify API]

In [14]:
# call function get_track_analysis() to extract tracks' analysis data
track_analysis = []
for track_id in track_data["track_id"]:
    ta = get_track_analysis(token, track_id)
    track_analysis.append(ta)

columns = list(track_analysis[0].keys())
rows = []
for row in track_analysis:
    rows.append(list(row.values()))

track_analysis_data = pd.DataFrame(data=rows, columns=columns)

In [15]:
track_analysis_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 26 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   num_samples                50 non-null     int64  
 1   duration                   50 non-null     float64
 2   sample_md5                 50 non-null     object 
 3   offset_seconds             50 non-null     int64  
 4   window_seconds             50 non-null     int64  
 5   analysis_sample_rate       50 non-null     int64  
 6   analysis_channels          50 non-null     int64  
 7   end_of_fade_in             50 non-null     float64
 8   start_of_fade_out          50 non-null     float64
 9   loudness                   50 non-null     float64
 10  tempo                      50 non-null     float64
 11  tempo_confidence           50 non-null     float64
 12  time_signature             50 non-null     int64  
 13  time_signature_confidence  50 non-null     float64
 

In [16]:
track_analysis_data.head()

,num_samples,duration,sample_md5,offset_seconds,window_seconds,analysis_sample_rate,analysis_channels,end_of_fade_in,start_of_fade_out,loudness,...,mode,mode_confidence,codestring,code_version,echoprintstring,echoprint_version,synchstring,synch_version,rhythmstring,rhythm_version
0,6857486,310.99710,,0,0,22050,1,0.45306,306.22476,-7.910,...,1,0.487,eJw9mwu23LgOA7fSS7D-0v43NlXgzbx3konVbluiSBCE2O...,3.15,eJzlnQmaLTdupbcUnMnlcNz_Evo_uLJSJVcwW856Vtnu_g...,4.12,eJxNWId1ZCEMbIUSAInUf2M3QX99z2vvD6A4GgnP2Vdv0e...,1.0,eJx9XAmyHCuMvEofAQQIuP_FhlxUz38mYiLssN_r6ioWLa...,1.0
1,4091892,185.57333,,0,0,22050,1,0.19152,179.80373,-6.124,...,0,0.480,eJxNmomV7LgRBF2hCbgP_x3biETP-yutVgM0CQJ1ZlZh9H...,3.15,eJzdnQmS5bhyZbdEAiCG5WDc_xL6XGcqmBVWxOv6oexSy2...,4.12,eJxlWIeN7TgMbMUliFTuv7HTBL79wAFer_2swDAckspss3...,1.0,eJxlmweSY8kNRK_CI5Q397-YkPlQv2dDEVr1kPwsA5tIgL...,1.0
2,4981536,225.92000,,0,0,22050,1,0.32141,221.66348,-7.335,...,1,0.575,eJxVmwmW5TiOBK_yj8B9uf_FxsypqM7p168yxC9RJAg4HI...,3.15,eJzlvQma5DiSpHslYgeOA4DA_Y_wflFmh0X5JOEdzyu7er...,4.12,eJxtWAlyBakOuwpHaC9s97_YtySTZGZ-VdcLocF4kWXT7t...,1.0,eJxlXFuC5CgOvEodgaeA-19siYdw9uxHT09Xpm1AUigUkq...,1.0
3,5113019,231.88295,,0,0,22050,1,0.47596,224.00871,-6.833,...,1,0.459,eJxNmwuSLCkOBK9SR0j-cP-LrXuon83urM00VFYCQgqFPn...,3.15,eJzlnQmyJDeSZK_k2IHjYL3_EeapBTuDlTKOL5xsziJTJa...,4.12,eJxNWQuC6zgIu0qOYMDf-19sjCTc2XbntYnjDwghqHsb7Y...,1.0,eJxtXAdy5DgM_IqewBz-_7FjB1Aa71XdnscaiSJBhEYDdM...,1.0
4,5439732,246.69986,,0,0,22050,1,0.39156,242.72398,-8.203,...,0,0.518,eJxNmwuWBacNRLfyltD8Yf8by72lsZ2cEyfN9KNBSKVSCf...,3.15,eJzVvQmS9LixpbslYgaWg3H_S3jf8ZAqpd-aiC7LW3b7mW...,4.12,eJxNmImRKzkMQ1NRCC3qzj-xJR7o-VtTHvch8QRByhHf-t...,1.0,eJxlXIe1IzkOTEUh0BPMP7FjGbD_7L23ZkZqQ8IWCqBqLW...,1.0


In [17]:
# explore each column's value
track_analysis_data.iloc[0]

num_samples                                                            6857486
duration                                                              310.9971
sample_md5                                                                    
offset_seconds                                                               0
window_seconds                                                               0
analysis_sample_rate                                                     22050
analysis_channels                                                            1
end_of_fade_in                                                         0.45306
start_of_fade_out                                                    306.22476
loudness                                                                 -7.91
tempo                                                                   143.99
tempo_confidence                                                         0.319
time_signature                                      

### Transform

Selected columns (desciption is available in attributes_detail/track_analysis_attr.txt)
- duration	
- loudness	
- tempo	
- time_signature	
- key	
- mode

In [18]:
track_analysis_data = track_analysis_data[['duration', 'loudness', 'tempo', 'time_signature', 'key', 'mode']]

In [26]:
# check null values
track_analysis_data.isnull().any().sum()

0

In [27]:
track_analysis_data.head()

,duration,loudness,tempo,time_signature,key,mode
0,310.99710,-7.910,143.990,4,9,1
1,185.57333,-6.124,158.078,4,6,0
2,225.92000,-7.335,97.025,4,1,1
3,231.88295,-6.833,116.982,4,2,1
4,246.69986,-8.203,123.031,4,5,0


In [21]:
# concat analysis data to the track data
track_data_concat = pd.concat([track_data, track_analysis_data] , axis=1)

In [22]:
track_data_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   track_id        50 non-null     object 
 1   track_name      50 non-null     object 
 2   href            50 non-null     object 
 3   popularity      50 non-null     int64  
 4   uri             50 non-null     object 
 5   release_date    50 non-null     object 
 6   album_id        50 non-null     object 
 7   album_name      50 non-null     object 
 8   duration        50 non-null     float64
 9   loudness        50 non-null     float64
 10  tempo           50 non-null     float64
 11  time_signature  50 non-null     int64  
 12  key             50 non-null     int64  
 13  mode            50 non-null     int64  
dtypes: float64(3), int64(4), object(7)
memory usage: 5.6+ KB


In [23]:
# check duplications
track_data_concat.duplicated().sum()

0

## Get track's audio features

### Extract [Spotify API]

In [24]:
# call function get_track_feature() to extract tracks' feature data
track_feature = []
for track_id in track_data["track_id"]:
    ta = get_track_feature(token, track_id)
    track_feature.append(ta)

# store the retrived data in the dataframe
columns = list(track_feature[0].keys())
rows = []
for row in track_feature:
    rows.append(list(row.values()))

track_feature_data = pd.DataFrame(data=rows, columns=columns)

In [28]:
track_feature_data.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.565,0.591,9,-7.910,1,0.0336,0.47300,0.052300,0.107,0.429,143.990,audio_features,4hsEFcCA7AL5hlKG4PRp2Z,spotify:track:4hsEFcCA7AL5hlKG4PRp2Z,https://api.spotify.com/v1/tracks/4hsEFcCA7AL5...,https://api.spotify.com/v1/audio-analysis/4hsE...,310997,4
1,0.600,0.760,6,-6.124,0,0.0452,0.16600,0.000041,0.189,0.519,158.078,audio_features,0o9zmvc5f3EFApU52PPIyW,spotify:track:0o9zmvc5f3EFApU52PPIyW,https://api.spotify.com/v1/tracks/0o9zmvc5f3EF...,https://api.spotify.com/v1/audio-analysis/0o9z...,185573,4
2,0.663,0.731,1,-7.335,1,0.0338,0.00892,0.050200,0.293,0.356,97.025,audio_features,7AMGgAPFczs3wJgMqu6Eqi,spotify:track:7AMGgAPFczs3wJgMqu6Eqi,https://api.spotify.com/v1/tracks/7AMGgAPFczs3...,https://api.spotify.com/v1/audio-analysis/7AMG...,225920,4
3,0.746,0.764,2,-6.833,1,0.0462,0.00803,0.000000,0.357,0.701,116.982,audio_features,51oc6MEsXTpnPn6GOw5VuP,spotify:track:51oc6MEsXTpnPn6GOw5VuP,https://api.spotify.com/v1/tracks/51oc6MEsXTpn...,https://api.spotify.com/v1/audio-analysis/51oc...,231883,4
4,0.769,0.716,5,-8.203,0,0.0429,0.11800,0.006590,0.158,0.660,123.031,audio_features,02tNuntKQsoou5T4O8meyh,spotify:track:02tNuntKQsoou5T4O8meyh,https://api.spotify.com/v1/tracks/02tNuntKQsoo...,https://api.spotify.com/v1/audio-analysis/02tN...,246700,4


In [25]:
track_feature_data.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.565,0.591,9,-7.910,1,0.0336,0.47300,0.052300,0.107,0.429,143.990,audio_features,4hsEFcCA7AL5hlKG4PRp2Z,spotify:track:4hsEFcCA7AL5hlKG4PRp2Z,https://api.spotify.com/v1/tracks/4hsEFcCA7AL5...,https://api.spotify.com/v1/audio-analysis/4hsE...,310997,4
1,0.600,0.760,6,-6.124,0,0.0452,0.16600,0.000041,0.189,0.519,158.078,audio_features,0o9zmvc5f3EFApU52PPIyW,spotify:track:0o9zmvc5f3EFApU52PPIyW,https://api.spotify.com/v1/tracks/0o9zmvc5f3EF...,https://api.spotify.com/v1/audio-analysis/0o9z...,185573,4
2,0.663,0.731,1,-7.335,1,0.0338,0.00892,0.050200,0.293,0.356,97.025,audio_features,7AMGgAPFczs3wJgMqu6Eqi,spotify:track:7AMGgAPFczs3wJgMqu6Eqi,https://api.spotify.com/v1/tracks/7AMGgAPFczs3...,https://api.spotify.com/v1/audio-analysis/7AMG...,225920,4
3,0.746,0.764,2,-6.833,1,0.0462,0.00803,0.000000,0.357,0.701,116.982,audio_features,51oc6MEsXTpnPn6GOw5VuP,spotify:track:51oc6MEsXTpnPn6GOw5VuP,https://api.spotify.com/v1/tracks/51oc6MEsXTpn...,https://api.spotify.com/v1/audio-analysis/51oc...,231883,4
4,0.769,0.716,5,-8.203,0,0.0429,0.11800,0.006590,0.158,0.660,123.031,audio_features,02tNuntKQsoou5T4O8meyh,spotify:track:02tNuntKQsoou5T4O8meyh,https://api.spotify.com/v1/tracks/02tNuntKQsoo...,https://api.spotify.com/v1/audio-analysis/02tN...,246700,4


In [382]:
# explore each column's value
track_feature_data.iloc[0]

danceability                                                    0.565
energy                                                          0.591
key                                                                 9
loudness                                                        -7.91
mode                                                                1
speechiness                                                    0.0336
acousticness                                                    0.473
instrumentalness                                               0.0523
liveness                                                        0.107
valence                                                         0.429
tempo                                                          143.99
type                                                   audio_features
id                                             4hsEFcCA7AL5hlKG4PRp2Z
uri                              spotify:track:4hsEFcCA7AL5hlKG4PRp2Z
track_href          

### Transform

In [30]:
#  rename the id to track_id
track_feature_data = track_feature_data.rename(columns={"id": "track_id"})

Selected columns (desciption is available in attributes_detail/track_feature_attr.txt)
- acousticness	
- energy	
- danceability	

In [1]:
track_feature_data = track_feature_data[['track_id', 'acousticness', 'energy','danceability']]

NameError: name 'track_feature_data' is not defined

In [32]:
track_feature_data.isnull().any().sum()

0

In [33]:
track_feature_data.head()

,track_id,acousticness,energy,instrumentalness,danceability,liveness,speechiness
0,4hsEFcCA7AL5hlKG4PRp2Z,0.47300,0.591,0.052300,0.565,0.107,0.0336
1,0o9zmvc5f3EFApU52PPIyW,0.16600,0.760,0.000041,0.600,0.189,0.0452
2,7AMGgAPFczs3wJgMqu6Eqi,0.00892,0.731,0.050200,0.663,0.293,0.0338
3,51oc6MEsXTpnPn6GOw5VuP,0.00803,0.764,0.000000,0.746,0.357,0.0462
4,02tNuntKQsoou5T4O8meyh,0.11800,0.716,0.006590,0.769,0.158,0.0429


In [42]:
# merge track feature data with track analysis data by track_id
track_data_merge = pd.merge(track_data_concat, track_feature_data, on='track_id')

### Transform final dataframe

In [43]:
track_data_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   track_id          50 non-null     object 
 1   track_name        50 non-null     object 
 2   href              50 non-null     object 
 3   popularity        50 non-null     int64  
 4   uri               50 non-null     object 
 5   release_date      50 non-null     object 
 6   album_id          50 non-null     object 
 7   album_name        50 non-null     object 
 8   duration          50 non-null     float64
 9   loudness          50 non-null     float64
 10  tempo             50 non-null     float64
 11  time_signature    50 non-null     int64  
 12  key               50 non-null     int64  
 13  mode              50 non-null     int64  
 14  acousticness      50 non-null     float64
 15  energy            50 non-null     float64
 16  instrumentalness  50 non-null     float64
 17 

In [44]:
# check null values
track_data_merge.isnull().any().sum()

0

In [45]:
# check duplications
track_data_merge.duplicated().sum()

0

In [46]:
# check duplications in track's name
track_data_merge.duplicated(subset='track_name').sum()

2

In [47]:
# find the tracks that have duplications
track_data_merge["track_name"][track_data_merge.duplicated(subset='track_name')]

43     きらり
47    damn
Name: track_name, dtype: object

In [48]:
# explore the data of track 'damn'
track_data_merge[track_data_merge['track_name'] == 'damn']

,track_id,track_name,href,popularity,uri,release_date,album_id,album_name,duration,loudness,tempo,time_signature,key,mode,acousticness,energy,instrumentalness,danceability,liveness,speechiness
6,7pTwW5yCKRx6OkByY53jbz,damn,https://api.spotify.com/v1/tracks/7pTwW5yCKRx6...,59,spotify:track:7pTwW5yCKRx6OkByY53jbz,2022-03-23,7Ip9X7pnkhJ4cwDoBnvneD,LOVE ALL SERVE ALL,259.02667,-7.955,129.982,4,7,1,0.00346,0.759,0.000038,0.707,0.1160,0.0329
47,0cEill2tVJ38caZlb3LyFy,damn,https://api.spotify.com/v1/tracks/0cEill2tVJ38...,41,spotify:track:0cEill2tVJ38caZlb3LyFy,2022-09-30,3xMDL5v9ftUpE1I7XPO8oR,damn,259.23795,-7.993,129.984,4,7,1,0.00348,0.758,0.000059,0.710,0.0999,0.0331


In [49]:
# explore the data of track 'きらり' (kirari)
track_data_merge[track_data_merge['track_name'] == 'きらり']

,track_id,track_name,href,popularity,uri,release_date,album_id,album_name,duration,loudness,tempo,time_signature,key,mode,acousticness,energy,instrumentalness,danceability,liveness,speechiness
3,51oc6MEsXTpnPn6GOw5VuP,きらり,https://api.spotify.com/v1/tracks/51oc6MEsXTpn...,70,spotify:track:51oc6MEsXTpnPn6GOw5VuP,2022-01-14,2OXwORzPU4tm1Skiv6l9KT,Kirari Remixes (Asia Edition),231.88295,-6.833,116.982,4,2,1,0.00803,0.764,0.0,0.746,0.357,0.0462
43,3ciqhcLmXP4hVGBD98QlEj,きらり,https://api.spotify.com/v1/tracks/3ciqhcLmXP4h...,56,spotify:track:3ciqhcLmXP4hVGBD98QlEj,2021-05-03,1fokTD5rDI2bHnFH1QSbPy,きらり,231.88295,-6.833,116.982,4,2,1,0.00803,0.764,0.0,0.746,0.357,0.0462


In this project, only the track data with the latest release date is kept. However, the values of all numeric columns are replaced with the mean values calculated from all rows of the track.

In [50]:
# calculate mean of numeric columns of track 'damn'
df_num = track_data_merge.select_dtypes(include=['number'])
damn_mean = df_num[track_data_merge['track_name']=='damn'].mean()
damn_mean

popularity           50.000000
duration            259.132310
loudness             -7.974000
tempo               129.983000
time_signature        4.000000
key                   7.000000
mode                  1.000000
acousticness          0.003470
energy                0.758500
instrumentalness      0.000048
danceability          0.708500
liveness              0.107950
speechiness           0.033000
dtype: float64

In [51]:
# calculate mean of numeric columns of track 'きらり' (kirari)
df_num = track_data_merge.select_dtypes(include=['number'])
kirari_mean = df_num[track_data_merge['track_name']=='きらり'].mean()
kirari_mean

popularity           63.00000
duration            231.88295
loudness             -6.83300
tempo               116.98200
time_signature        4.00000
key                   2.00000
mode                  1.00000
acousticness          0.00803
energy                0.76400
instrumentalness      0.00000
danceability          0.74600
liveness              0.35700
speechiness           0.04620
dtype: float64

In [52]:
# drop duplications - keep first (latest release date)
df_drop = track_data_merge.drop_duplicates(subset='track_name', keep='first')

In [53]:
# replace numeric values of track 'damn' with its mean
index_damn = df_drop[df_drop['track_name'] == 'damn'].index
index_damn 
df_drop.loc[index_damn , df_num.columns] = [damn_mean]
df_drop[df_drop['track_name'] == 'damn']


/var/folders/3y/503q0vhn7bx6zn631q4t5wn40000gn/T/ipykernel_14797/3245653463.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[50.0]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_drop.loc[index_damn , df_num.columns] = [damn_mean]
/var/folders/3y/503q0vhn7bx6zn631q4t5wn40000gn/T/ipykernel_14797/3245653463.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[4.0]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_drop.loc[index_damn , df_num.columns] = [damn_mean]
/var/folders/3y/503q0vhn7bx6zn631q4t5wn40000gn/T/ipykernel_14797/3245653463.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[7.0]' has dtype incompatible with int64, please explicitly cast

,track_id,track_name,href,popularity,uri,release_date,album_id,album_name,duration,loudness,tempo,time_signature,key,mode,acousticness,energy,instrumentalness,danceability,liveness,speechiness
6,7pTwW5yCKRx6OkByY53jbz,damn,https://api.spotify.com/v1/tracks/7pTwW5yCKRx6...,50.0,spotify:track:7pTwW5yCKRx6OkByY53jbz,2022-03-23,7Ip9X7pnkhJ4cwDoBnvneD,LOVE ALL SERVE ALL,259.13231,-7.974,129.983,4.0,7.0,1.0,0.00347,0.7585,0.000048,0.7085,0.10795,0.033


In [54]:
# replace numeric values of track 'きらり' (kirari) with its mean
index_kirari = df_drop[df_drop['track_name'] == 'きらり'].index
index_kirari 
df_drop.loc[index_kirari  , df_num.columns] = [kirari_mean]
df_drop[df_drop['track_name'] == 'きらり']

,track_id,track_name,href,popularity,uri,release_date,album_id,album_name,duration,loudness,tempo,time_signature,key,mode,acousticness,energy,instrumentalness,danceability,liveness,speechiness
3,51oc6MEsXTpnPn6GOw5VuP,きらり,https://api.spotify.com/v1/tracks/51oc6MEsXTpn...,63.0,spotify:track:51oc6MEsXTpnPn6GOw5VuP,2022-01-14,2OXwORzPU4tm1Skiv6l9KT,Kirari Remixes (Asia Edition),231.88295,-6.833,116.982,4.0,2.0,1.0,0.00803,0.764,0.0,0.746,0.357,0.0462


In [55]:
track_data_merge = df_drop.copy()
track_data_merge.info()

<class 'pandas.core.frame.DataFrame'>
Index: 48 entries, 0 to 49
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   track_id          48 non-null     object 
 1   track_name        48 non-null     object 
 2   href              48 non-null     object 
 3   popularity        48 non-null     float64
 4   uri               48 non-null     object 
 5   release_date      48 non-null     object 
 6   album_id          48 non-null     object 
 7   album_name        48 non-null     object 
 8   duration          48 non-null     float64
 9   loudness          48 non-null     float64
 10  tempo             48 non-null     float64
 11  time_signature    48 non-null     float64
 12  key               48 non-null     float64
 13  mode              48 non-null     float64
 14  acousticness      48 non-null     float64
 15  energy            48 non-null     float64
 16  instrumentalness  48 non-null     float64
 17  danc

In [56]:
# download cleaned data as .csv file
track_data_merge.to_csv(f"track_data_cleaned.csv")

### Transform data for visualization

In [57]:
track_data_visualize = track_data_merge.copy()

In [58]:
# following the definition provided by Spotify 
Mode = {
    0 : "Minor",
    1 : "Major"
}

Key = {
    -1: "No Key",
    0: "C",
    1: "C#/Db",
    2: "D",
    3: "D#/Eb",
    4: "E",
    5: "F",
    6: "F#/Gb",
    7: "G",
    8: "G#/Ab",
    9: "A",
    10: "A#/Bb",
    11: "B"
}

In [59]:
# map mode and key to defined dictionary
track_data_visualize["mode"] = track_data_merge["mode"].map(Mode)
track_data_visualize["key"] = track_data_merge["key"].map(Key)

In [60]:
# time_signature (i.e. 3 -> 3/4)
track_data_visualize["time_signature"] = track_data_merge["time_signature"].apply(lambda x: int(x)).astype(str) + "/4"

In [61]:
track_data_visualize.head()

,track_id,track_name,href,popularity,uri,release_date,album_id,album_name,duration,loudness,tempo,time_signature,key,mode,acousticness,energy,instrumentalness,danceability,liveness,speechiness
0,4hsEFcCA7AL5hlKG4PRp2Z,Michi Teyu Ku (Overflowing),https://api.spotify.com/v1/tracks/4hsEFcCA7AL5...,69.0,spotify:track:4hsEFcCA7AL5hlKG4PRp2Z,2024-03-14,06qRSgIo9l4Gf7ACV6JZQq,Michi Teyu Ku (Overflowing),310.99710,-7.910,143.990,4/4,A,Major,0.47300,0.591,0.052300,0.565,0.107,0.0336
1,0o9zmvc5f3EFApU52PPIyW,Shinunoga E-Wa,https://api.spotify.com/v1/tracks/0o9zmvc5f3EF...,78.0,spotify:track:0o9zmvc5f3EFApU52PPIyW,2020-05-20,1OojCidx0eoPKch2M0Kz31,HELP EVER HURT NEVER,185.57333,-6.124,158.078,4/4,F#/Gb,Minor,0.16600,0.760,0.000041,0.600,0.189,0.0452
2,7AMGgAPFczs3wJgMqu6Eqi,Matsuri,https://api.spotify.com/v1/tracks/7AMGgAPFczs3...,68.0,spotify:track:7AMGgAPFczs3wJgMqu6Eqi,2022-03-23,7Ip9X7pnkhJ4cwDoBnvneD,LOVE ALL SERVE ALL,225.92000,-7.335,97.025,4/4,C#/Db,Major,0.00892,0.731,0.050200,0.663,0.293,0.0338
3,51oc6MEsXTpnPn6GOw5VuP,きらり,https://api.spotify.com/v1/tracks/51oc6MEsXTpn...,63.0,spotify:track:51oc6MEsXTpnPn6GOw5VuP,2022-01-14,2OXwORzPU4tm1Skiv6l9KT,Kirari Remixes (Asia Edition),231.88295,-6.833,116.982,4/4,D,Major,0.00803,0.764,0.000000,0.746,0.357,0.0462
4,02tNuntKQsoou5T4O8meyh,Hana,https://api.spotify.com/v1/tracks/02tNuntKQsoo...,62.0,spotify:track:02tNuntKQsoou5T4O8meyh,2023-10-12,0v6vQ9tMopUTccn0wUvzQD,Hana,246.69986,-8.203,123.031,4/4,F,Minor,0.11800,0.716,0.006590,0.769,0.158,0.0429


In [62]:
track_data_visualize.info()

<class 'pandas.core.frame.DataFrame'>
Index: 48 entries, 0 to 49
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   track_id          48 non-null     object 
 1   track_name        48 non-null     object 
 2   href              48 non-null     object 
 3   popularity        48 non-null     float64
 4   uri               48 non-null     object 
 5   release_date      48 non-null     object 
 6   album_id          48 non-null     object 
 7   album_name        48 non-null     object 
 8   duration          48 non-null     float64
 9   loudness          48 non-null     float64
 10  tempo             48 non-null     float64
 11  time_signature    48 non-null     object 
 12  key               48 non-null     object 
 13  mode              48 non-null     object 
 14  acousticness      48 non-null     float64
 15  energy            48 non-null     float64
 16  instrumentalness  48 non-null     float64
 17  danc

In [63]:
# download cleaned data as .csv file
track_data_visualize.to_csv(f"track_data_cleaned_visual.csv")